<a href="https://colab.research.google.com/github/pauloassumpcao/mnist_fashion_WiSARD/blob/main/mnist_fashion_wisardpkg_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wisardpkg

In [2]:
import csv
import wisardpkg as wp
from wisardpkg import ClusWisard
from sklearn.metrics import accuracy_score

## Carregando os dados (fashion_mnist)

In [3]:
with open("/content/sample_data/fashion-mnist_train.csv", 'r') as f:
    mnist_train = list(csv.reader(f, delimiter=","))

with open("/content/sample_data/fashion-mnist_test.csv", 'r') as f:
    mnist_test = list(csv.reader(f, delimiter=","))

labels_train = []
inputs_train = []
labels_test = []
inputs_test = []
for i in range(1, len(mnist_train)):
    labels_train.append(mnist_train[i][0])
    inputs_train.append(mnist_train[i][1:])

print("Label train: ", labels_train[0:10])
print("Input train: ", inputs_train[0:10])

for i in range(1, len(mnist_test)):
    labels_test.append(mnist_test[i][0])
    inputs_test.append(mnist_test[i][1:])

print("Label test: ", labels_test[0:10])
print("Input test: ", inputs_test[0:10])

Label train:  ['2', '9', '6', '0', '3', '4', '4', '5', '4', '8']
Input train:  [['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '4', '0', '0', '0', '0', '0', '62', '61', '21', '29', '23', '51', '136', '61', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '88', '201', '228', '225', '255', '115', '62', '137', '255', '235', '222', '255', '135', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '47', '252', '234', '238', '224', '215', '215', '229', '108', '180', '207', '214', '224', '231', '249', '254', '45', '0', '0', '0', '0', '0', '0', '0', '0', '1',

## Binarizando

In [4]:
def binarizar(inputs, threshold):
    #Convert an array elements in binary elements
    for i in range(len(inputs)):
        for k in range(len(inputs[i])):
          inputs[i][k] = int(inputs[i][k])
          if inputs[i][k] < threshold:
              inputs[i][k] = 0
          else:
              inputs[i][k] = 1

binarizar(inputs_train, 128)
binarizar(inputs_test, 128)


## Aplicando a WISARD

In [5]:
addressSize = 32 # número de bits de endereçamento na memória ram
ignoreZero  = False # faz com que as rams ignorem o endereço 0
verbose = True # imprime o andamento do métodos train e classify

minScore = 0.1 # pontuação mínima do processo de treinamento
threshold = 10 # limite de ciclos de treinamento por discriminador
discriminatorLimit = 5 # limite de discrimidadores por cluster

In [6]:
wisard = wp.Wisard(addressSize, ignoreZero=ignoreZero, verbose=verbose)
wisard.train(inputs_train, labels_train)
out_wisard = wisard.classify(inputs_test)
print(out_wisard)

['0', '1', '2', '9', '3', '6', '8', '6', '5', '0', '3', '2', '4', '6', '8', '5', '4', '3', '6', '9', '4', '4', '2', '1', '5', '9', '8', '6', '4', '1', '7', '7', '7', '8', '1', '3', '9', '6', '8', '3', '4', '0', '6', '2', '2', '0', '3', '3', '2', '3', '2', '4', '9', '3', '0', '9', '9', '4', '0', '3', '4', '5', '4', '2', '1', '1', '0', '9', '5', '2', '7', '3', '4', '0', '5', '7', '1', '6', '1', '3', '9', '0', '1', '4', '4', '7', '9', '4', '1', '6', '3', '6', '4', '6', '5', '3', '4', '7', '7', '3', '9', '4', '9', '6', '8', '2', '3', '8', '2', '7', '5', '5', '3', '2', '7', '5', '0', '2', '7', '3', '9', '5', '4', '4', '7', '0', '5', '9', '0', '3', '1', '5', '9', '2', '9', '6', '4', '2', '0', '4', '3', '3', '3', '4', '6', '2', '0', '0', '1', '3', '6', '3', '3', '1', '9', '3', '3', '3', '9', '5', '3', '7', '7', '3', '4', '2', '0', '9', '7', '6', '4', '8', '9', '0', '8', '4', '6', '3', '7', '9', '5', '1', '3', '9', '1', '7', '5', '4', '0', '7', '4', '7', '1', '6', '3', '0', '0', '4', '6', '9',

## Avialiando a acuracia da WISARD

In [7]:
ac_wisard = accuracy_score(out_wisard, labels_test)
print("Acuracia WISARD: ", ac_wisard)

Acuracia WISARD:  0.7583


## Aplicando a ClusWISARD

In [8]:
clus = ClusWisard(addressSize, minScore, threshold, discriminatorLimit, verbose=True)
clus.train(inputs_train, labels_train)
out_clus = clus.classify(inputs_test)
print(out_clus)

['0', '1', '2', '9', '3', '6', '8', '6', '5', '0', '3', '4', '4', '4', '8', '5', '2', '3', '6', '2', '4', '4', '2', '1', '5', '7', '8', '7', '4', '1', '9', '7', '9', '8', '1', '5', '9', '8', '0', '8', '2', '0', '2', '6', '2', '0', '3', '3', '2', '3', '6', '4', '9', '3', '0', '9', '9', '4', '6', '3', '2', '5', '2', '6', '1', '1', '0', '9', '5', '2', '7', '6', '2', '0', '5', '7', '1', '2', '1', '3', '9', '0', '1', '2', '2', '5', '9', '4', '1', '6', '3', '6', '2', '6', '0', '0', '2', '7', '7', '3', '9', '0', '9', '6', '8', '2', '3', '8', '2', '5', '5', '5', '4', '2', '7', '5', '0', '0', '7', '3', '9', '5', '6', '4', '7', '0', '5', '5', '0', '3', '1', '5', '9', '4', '9', '6', '4', '4', '0', '4', '3', '3', '3', '4', '2', '4', '0', '0', '1', '3', '4', '3', '3', '1', '9', '3', '3', '0', '9', '5', '2', '7', '7', '3', '2', '2', '0', '2', '5', '2', '2', '8', '9', '0', '2', '2', '2', '5', '7', '9', '5', '1', '3', '9', '1', '5', '5', '2', '0', '7', '4', '9', '1', '2', '6', '0', '2', '6', '0', '9',

##Avaliando a acuracia da ClusWISARD

In [9]:
ac_clus = accuracy_score(out_clus, labels_test)
print("Acuracia ClusWISARD: ", ac_clus)


Acuracia ClusWISARD:  0.7585


## Melhor resultado

In [10]:
if ac_clus > ac_wisard:
  print("ClusWISARD teve o melhor resultado: ", ac_clus)
else:
  print("WISARD teve o melhor resultado: ", ac_wisard)

ClusWISARD teve o melhor resultado:  0.7585
